## Task 1 - Explain ciprofloxacin resistance

**A common mutation causing ciprofloxacin resistance is found in the *gyrA* and results in the following amino acid change - S91F. Can you confirm if this is present in the sequence data, using the mapped data, and find another resistance determinant?**

This relies on using python to identify the gene in the whole genome andconvert DNA sequence to protein sequences, so we can then compare the reference genome with our genome of interest to look for differences.

### Tip 1

Find the location of the *gyrA* gene in the reference file using the NCBI website - https://www.ncbi.nlm.nih.gov/nuccore/NC_011035.1?report=graph - use the find function on the webpage

- Coordinates of the *gyrA* gene on [NCBI](https://www.ncbi.nlm.nih.gov/nuccore/NC_011035.1?report=graph): 1051396 to 1054146
- In Python: 1051395 to 1054145.

![NCBI Screenshot](task1_tip1.png)

### Tip 2

Use biopython's `SeqIO.read()` function to read in the mapped fasta file `cdt-tutorial/super_gonorrhoea/super_gc_mapped.fa` (https://biopython.org/wiki/SeqIO)

**Change path.**

In [9]:
from Bio import SeqIO
super_gc_mapped_fa = SeqIO.read('../super_gonorrhoea/super_gc_mapped.fa', 'fasta')

### Tip 3

You can will need to extract the DNA sequence of the gene using it's coordinates within the whole genome (remember that these number from zero in python, but from one on the NCBI website!)

In [7]:
super_gc_GyrA_extract = super_gc_mapped_fa[1051395:1054146]
print(super_gc_GyrA_extract)

ID: OxfordGC_R00000419_R00000419
Name: OxfordGC_R00000419_R00000419
Description: OxfordGC_R00000419_R00000419 Software:/home/compass/PIPELINE/mmmPipeline/compass/g4_basecall.py Version:1.0.1
Number of features: 0
Seq('ATGACCGACGCAACCATCCGCCACGACCACAAATTCGCCCTCGAAACCCTGCCC...TGA', SingleLetterAlphabet())


### Tip 4

You will need to translate the DNA sequences to a protein sequence, `seq.translate()` in biopython (for *gyrA* the gene is in the same direction as the DNA is numbered, if it were not you would need to generate the reverse complement sequence first), if this works you should end up with a string of amino acids represented as single letters and ending with a stop codon shown as an asterisk.

In [8]:
super_gc_GyrA = super_gc_GyrA_extract.translate()
print(super_gc_GyrA)

ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('MTDATIRHDHKFALETLPVSLEDEMRKSYLDYAMSVIVGRALPDVRDGLKPVHR...EN*', HasStopCodon(ExtendedIUPACProtein(), '*'))


### Tip 5

Compare the amino acid sequences from the reference (`cdt-tutorial/super_gonorrhoea/reference.fa` ) with the sequence from the case using python, does the reference share any mutations or have any different mutations in *gyrA* (hint the reference sequence is also ciprofloxacin resistant)?

**Change path.**

In [19]:
# explore 90-th position
super_gc_GyrA[90]

'F'

We have 'F'on the 91st (90th from 0) place, so we confirm the amino acid change - S91F.

**S91F** means S->F at position 91. Indeed, we have 'F' at position 91. 'S' (Serine) and 'F' (Phenylalanine), in this case, are amino-acids in their [one-letter codes](https://en.wikipedia.org/wiki/Amino_acid#:~:text=Amino%20acids%20are%20the%20structural,to%20two%20neighboring%20amino%20acids.).

In [27]:
# reference

reference_mapped_fa = SeqIO.read('../super_gonorrhoea/reference.fa', 'fasta')
reference_GyrA = reference_gc_mapped_fa[1051395:1054146].translate()
print(reference_GyrA)

ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('MTDATIRHDHKFALETLPVSLEDEMRKSYLDYAMSVIVGRALPDVRDGLKPVHR...EN*', HasStopCodon(ExtendedIUPACProtein(), '*'))


In [53]:
import numpy as np

inconsistent_idxs = np.array((reference_GyrA)) != np.array((super_gc_GyrA))
print(sum(inconsistent_idxs), 'inconsistent index(es)')

(np.arange(1, len(reference_GyrA)+1)[inconsistent_idxs],
np.array(super_gc_GyrA)[inconsistent_idxs],
np.array(reference_GyrA)[inconsistent_idxs])

1 inconsistent index(es)


(array([95]), array(['A'], dtype='<U1'), array(['G'], dtype='<U1'))

In [54]:
for i, (aa1, aa2) in enumerate(zip(super_gc_GyrA, reference_GyrA)):
    if aa1!=aa2:
        print(i+1, aa1, aa2)

95 A G


## Task 2 - Explain the tetracycline resistance
**Tetracycline resistance is conferred by gain of a new gene, *tetM*, carried on a plasmid. Is this gene present?**  
To do this we will use a tool called BLAST. This is an optimised algorithm for searching for one or more sequences (the query) within a database of sequences. In this example we will treat the gene we are trying to find as the query and the database will be the contigs that make up the assembly of our genome of interest.

I suggest you read the Process and Algorithm sections of the wikipedia article on BLAST before continuing - as this will help with parsing the output generated - https://en.wikipedia.org/wiki/BLAST_(biotechnology).

### Tip1
Use the de novo assembly provided `super_gc_contigs.fa`

### Tip 2
You will need to create blast database files for the de novo assembly with this command:  
```makeblastdb -dbtype nucl -in super_gc_contigs.fa```

This can be done using this snippet in python to run the command within the notebook:
```
import os
cmd = 'makeblastdb -dbtype nucl -in cdt-tutorial/super_gonorrhoea/super_gc_contigs.fa'
os.system(cmd)
```

**Change path.**

In [56]:
import os
cmd = 'makeblastdb -dbtype nucl -in ../super_gonorrhoea/super_gc_contigs.fa'
os.system(cmd)

0

In [62]:
# check that we generated `super_gc_contigs.fa.n*` files
os.listdir('../super_gonorrhoea/')

['super_gc_contigs.fa.ntf',
 'tetM.fa',
 'super_gc_contigs.fa.ndb',
 'penA_alleles.fa',
 'super_gc_contigs.fa.nsq',
 'super_gc_contigs.fa.nin',
 'reference.fa',
 'super_gc_mapped.fa',
 '23s_reference.fa',
 'super_gc_contigs.fa',
 'super_gc_contigs.fa.nto',
 'super_gc_contigs.fa.nhr',
 'super_gc_contigs.fa.not']

### Tip 3
Use the biopython blast module to run a blast command that searches for the tetM gene, to get you started:  

```
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from io import StringIO

os.chdir('cdt-tutorial/super_gonorrhoea/')

geneFile = 'tetM.fa'
assemblyFile = 'super_gc_contigs.fa'

cline = NcbiblastnCommandline( query=geneFile, 
         db=assemblyFile, 
         evalue=0.01, 
         outfmt=5
         )
stdout, stderr = cline()
fp = StringIO( stdout )
blast_records = [r for r in NCBIXML.parse( fp )]
```
This snippet will run blast, saving the output in XML format, this is then passed back to python. You should be able interrogate the `blast_records` object to look for any matches found. Details of the contents of the object can be found here - http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc93.

In [63]:
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from io import StringIO

os.chdir('../super_gonorrhoea/')

geneFile = 'tetM.fa'
assemblyFile = 'super_gc_contigs.fa'

cline = NcbiblastnCommandline( query=geneFile, 
         db=assemblyFile, 
         evalue=0.01, 
         outfmt=5
         )
stdout, stderr = cline()
fp = StringIO( stdout )
blast_records = [r for r in NCBIXML.parse( fp )]

### Tip 4
- Did you find an **alignment match**?
- How much of the total length of the tetM gene was matched?
- How similar was it to the tetM sequence in `tetM.fa`?

In [66]:
# from answers
for record in blast_records:
    print('%s has %s alignments\n'%(record.query, len(record.alignments)))
    if len(record.alignments)>0:
        hits = []
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                print('tetM length: %s'%len(hsp.query))
                print('Alignment length: %s'%hsp.align_length)
                print('Number of identities: %s'%hsp.identities)
                print('Number of gaps: %s'%hsp.gaps)

tetM has 1 alignments

tetM length: 1932
Alignment length: 1932
Number of identities: 1932
Number of gaps: 0


**Full match.**

## Task 3 - Explain the resistance to ceftriaxone
**Here we build on the example above to look for the best match to the *penA* gene using BLAST.**  
A number of mechanisms can change how susceptible N. gonorrhoeae is to ceftriaxone, however the gene *penA* is the most important. Although a few SNPs within the *penA* gene, e.g. A311V, T316P and T483S, the rest of the gene is very diverse (it was generated by recombination events with similar genes in other Neisseria species), and so mapping approaches do not work well for finding these variants - as the sequence is too different to the reference to map reliably. We therefore use a de novo assembly and a database of know *penA* sequences or alleles. We will compare our sequence of interest to this list of alleles. 

### Tip 1
Start as in Task 2, using the assembly as the blast database and this time using the list of *penA* alleles as the query `penA_alleles.fa` (if you want you can also try this task in reverse using the list of *penA* alleles as the database, to do this you'll need to run makeblastdb on the command line as above)

In [72]:
geneFile = 'penA_alleles.fa'
assemblyFile = 'super_gc_contigs.fa'

cline = NcbiblastnCommandline( query=geneFile, 
         db=assemblyFile, 
         evalue=0.01, 
         outfmt=5
         )
stdout, stderr = cline()
fp = StringIO( stdout )
blast_records = [r for r in NCBIXML.parse( fp )]

Print the hits.

In [73]:
for record in blast_records:
    print('%s has %s alignments'%(record.query, len(record.alignments)))
    if len(record.alignments)>0:
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                print('penA length: %s'%len(hsp.query))
                print('Alignment length: %s'%hsp.align_length)
                print('Number of identities: %s'%hsp.identities)
                print('Number of gaps: %s\n'%hsp.gaps)

penA_1.001 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1585
Number of gaps: 14

penA_1.002 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1583
Number of gaps: 14

penA_1.003 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1584
Number of gaps: 14

penA_1.004 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1584
Number of gaps: 14

penA_1.005 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1584
Number of gaps: 14

penA_1.006 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1583
Number of gaps: 14

penA_1.007 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1583
Number of gaps: 14

penA_1.008 has 1 alignments
penA length: 1756
Alignment length: 1756
Number of identities: 1585
Number of gaps: 14

penA_1.009 has 1 alignments
penA length: 1756
Alignment length: 1756
Num

### Tip 2
You will get multiple hits, you need to define which hit is the closest match

In [78]:
hsp_summary_best = None

# iterate through
for record in blast_records:
    if len(record.alignments)>0:
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                
                # initialize best
                if not hsp_summary_best:
                    hsp_summary_best = [record.query, hsp.align_length, hsp.identities, hsp.gaps, hsp.sbjct]
                
                # record current
                hsp_summary_current = [record.query, hsp.align_length, hsp.identities, hsp.gaps, hsp.sbjct]
                
                # update best if current is more alligned
                if hsp_summary_current[2] > hsp_summary_best[2]:
                    hsp_summary_best = hsp_summary_current

print(hsp_summary_best)

['penA_60.001', 1749, 1749, 0, 'ATGTTGATTAAAAGCGAATATAAGCCCCGGATGCTGCCCAAAGAAGAGCAGGTCAAAAAGCCGATGACCAGTAACGGACGGATTAGCTTCGTCCTGATGGCAATGGCGGTCTTGTTTGCCTGTCTGATTGCCCGCGGGCTGTATCTGCAGACGGTAACGTATAACTTTTTGAAAGAACAGGGCGACAACCGGATTGTGCGGACTCAAGCATTGCCGGCTACACGCGGTACGGTTTCGGACCGGAACGGTGCGGTTTTGGCGTTGAGCGCGCCGACGGAGTCCCTGTTTGCCGTGCCTAAAGATATGAAGGAAATGCCGTCTGCCGCCCAATTGGAACGCCTGTCCGAGCTTGTCGATGTGCCGGTCGATGTTTTGAGGAACAAACTCGAACAGAAAGGCAAGTCGTTTATTTGGATCAAGCGGCAGCTCGATCCCAAGGTTGCCGAAGAGGTCAAAGCCTTGGGTTTGGAAAACTTTGTATTTGAAAAAGAATTAAAACGCCATTACCCGATGGGCAACCTGTTTGCACACGTCATCGGATTTACCGATATTGACGGCAAAGGTCAGGAAGGTTTGGAACTTTCGCTTGAAGACAGCCTGTATGGCGAAGACGGCGCGGAAGTTGTTTTGCGGGACCGGCAGGGCAATATTGTGGACAGCTTGGACTCCCCGCGCAATAAAGCACCGCAAAACGGCAAAGACATCATCCTTTCCCTCGATCAGAGGATTCAGACCTTGGCCTATGAAGAGTTGAACAAGGCGGTCGAATACCATCAGGCAAAAGCCGGAACGGTGGTGGTTTTGGATGCCCGCACGGGGGAAATCCTCGCCTTGGCCAATACGCCCGCCTACGATCCCAACAGACCCGGCCGGGCAGACAGCGAACAGCGGCGCAACCGTGCCGTTACCGACATGATCGAACCTGGTTCTGTCATGAAGCCGTTTACCATTGCCAAAGCATTGGATTC

### Tip 3
Once you find which hit is the closest match, check to see if there is an exact match, and translate the sequence to see which of these amino acid substitutions is present - A311V, T316P and T483S (for this last part you may wish to turn the `hsp.sbjct` text from the BLAST output back into a BioPython sequence object - this will help you - https://biopython.org/DIST/docs/api/Bio.SeqRecord.SeqRecord-class.html).

- A311V
- T316P
- T483S 

In [79]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

seq = SeqRecord(Seq(hsp_summary_best[4]), name=hsp_summary_best[0])
print(seq)

ID: <unknown id>
Name: penA_60.001
Description: <unknown description>
Number of features: 0
Seq('ATGTTGATTAAAAGCGAATATAAGCCCCGGATGCTGCCCAAAGAAGAGCAGGTC...TAA')


In [82]:
protein = seq.translate()
print(protein[310], protein[315], protein[482], sep='\n')

V
T
S


- A311**V**: present
- T316**P**: absent
- T483**S**: present